In [1]:
import torch
import torch.nn as nn

from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
batch_size = 512 if device.type == "cuda" else 128
lr = 0.01
epochs = 20

In [4]:
train_data = datasets.MNIST(root="data", train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.MNIST(root="data", train=False, download=True, transform=transforms.ToTensor())

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [5]:
class Teacher(nn.Module):
  def __init__(self, dropout=0.2):
    super().__init__()
    self.model = nn.Sequential(
      nn.Dropout(p=dropout),
      nn.Linear(784, 1024),
      nn.ReLU(),
      nn.Linear(1024, 1024),
      nn.ReLU(),
      nn.Linear(1024, 10),
    )

  def forward(self, x):
    x = torch.flatten(x, 1)
    return self.model(x)

In [6]:
teacher = Teacher().to(device)
optimizer = torch.optim.Adam(teacher.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
loss_fn = nn.CrossEntropyLoss()

In [7]:
acc = -1.
for e in range(epochs):
  teacher.train()
  for x, y in (pbar := tqdm(train_dataloader)):
    x, y = x.to(device), y.to(device)
    optimizer.zero_grad(set_to_none=True)

    out = teacher(x)
    loss = loss_fn(out, y)
    loss.backward()
    optimizer.step()

    pbar.set_description(f"Epoch {e+1}/{epochs} Loss: {loss.item():.4f}")
    pbar.set_postfix(acc=acc)

  teacher.eval()
  with torch.no_grad():
    total = 0
    correct = 0
    for x, y in test_dataloader:
      x, y = x.to(device), y.to(device)
      out = teacher(x)
      _, pred = torch.max(out, dim=1)
      total += y.size(0)
      correct += (pred == y).sum().item()
  acc = (correct / total)
  scheduler.step()
print(f"Final accuracy: {acc*100:.2f}%")

Epoch 20/20 Loss: 0.0005: 100%|██████████| 118/118 [00:08<00:00, 13.23it/s, acc=0.983]


Final accuracy: 98.23%


In [8]:
class Student(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
      nn.Linear(784, 32),
      nn.ReLU(),
      nn.Linear(32, 10),
    )

  def forward(self, x):
    x = torch.flatten(x, 1)
    return self.model(x)

In [9]:
student = Student().to(device)
optimizer = torch.optim.Adam(student.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

In [10]:
acc = -1.
for e in range(epochs):
  student.train()
  for x, y in (pbar := tqdm(train_dataloader)):
    x, y = x.to(device), y.to(device)
    optimizer.zero_grad(set_to_none=True)

    out = student(x)
    loss = loss_fn(out, y)
    loss.backward()
    optimizer.step()

    pbar.set_description(f"Epoch {e+1}/{epochs} Loss: {loss.item():.4f}")
    pbar.set_postfix(acc=acc)

  student.eval()
  with torch.no_grad():
    total = 0
    correct = 0
    for x, y in test_dataloader:
      x, y = x.to(device), y.to(device)
      out = student(x)
      _, pred = torch.max(out, dim=1)
      total += y.size(0)
      correct += (pred == y).sum().item()
  acc = (correct / total)
  scheduler.step()
print(f"Final accuracy: {acc*100:.2f}%")

Epoch 20/20 Loss: 0.0618: 100%|██████████| 118/118 [00:08<00:00, 13.51it/s, acc=0.968]


Final accuracy: 96.91%


In [11]:
teacher.eval()
student = Student().to(device)
optimizer = torch.optim.Adam(student.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

kl_div_loss = nn.KLDivLoss(reduction="batchmean", log_target=True)

temperature = 2
soft_targets_weight = temperature ** 2
hard_targets_weight = 0.5

In [12]:
acc = -1.
for e in range(epochs):
  student.train()
  for x, y in (pbar := tqdm(train_dataloader)):
    x, y = x.to(device), y.to(device)
    optimizer.zero_grad(set_to_none=True)

    with torch.no_grad():
      teacher_logits = teacher(x)
    out = student(x)

    soft_targets = (teacher_logits / temperature).log_softmax(dim=-1)
    soft_prob = (out / temperature).log_softmax(dim=-1)

    soft_targets_loss = kl_div_loss(soft_prob, soft_targets)
    hard_targets_loss = loss_fn(out, y)

    loss = soft_targets_weight * soft_targets_loss + hard_targets_weight * hard_targets_loss
    loss.backward()
    optimizer.step()

    pbar.set_description(f"Epoch {e+1}/{epochs} Loss: {loss.item():.4f}")
    pbar.set_postfix(acc=acc)

  student.eval()
  if e % 5 == 4:
    with torch.no_grad():
      total = 0
      correct = 0
      for x, y in test_dataloader:
        x, y = x.to(device), y.to(device)
        out = student(x)
        _, pred = torch.max(out, dim=1)
        total += y.size(0)
        correct += (pred == y).sum().item()
  acc = (correct / total)
  scheduler.step()
print(f"Final accuracy: {acc*100:.2f}%")

Epoch 20/20 Loss: 0.1311: 100%|██████████| 118/118 [00:09<00:00, 12.15it/s, acc=0.971]


Final accuracy: 97.08%
